In [3]:
!pip install Pillow reportlab tqdm


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# URL cơ sở
base_url = "http://dlib.ptit.edu.vn/flowpaper/services/view.php?doc=81108511478093615990372473954145540650&format=jpg&subfolder=81/10/85/"

# Thư mục để lưu ảnh
output_dir = "downloaded_images"
os.makedirs(output_dir, exist_ok=True)

# Hàm tải xuống một ảnh
def download_image(page):
    url = f"{base_url}&page={page}"
    response = requests.get(url)
    if response.status_code == 200:
        with open(f"{output_dir}/page_{page}.jpg", "wb") as f:
            f.write(response.content)
        return True
    return False

# Tải xuống ảnh sử dụng đa luồng
def download_images(start_page, end_page):
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(download_image, range(start_page, end_page + 1)), total=end_page - start_page + 1))
    return results

# Chạy script
start_page = 1
end_page = 114

print(f"Bắt đầu tải xuống {end_page - start_page + 1} ảnh...")
results = download_images(start_page, end_page)

success_count = sum(results)
print(f"Đã tải xuống thành công {success_count} ảnh.")
print(f"Thất bại: {len(results) - success_count} ảnh.")
print(f"Ảnh đã được lưu trong thư mục: {output_dir}")

Bắt đầu tải xuống 114 ảnh...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:46<00:00,  2.43it/s]

Đã tải xuống thành công 114 ảnh.
Thất bại: 0 ảnh.
Ảnh đã được lưu trong thư mục: downloaded_images


In [2]:
import os
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.utils import ImageReader
from tqdm import tqdm

def merge_images_to_pdf(input_dir, output_pdf):
    # Lấy danh sách tất cả các file jpg trong thư mục
    image_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')],
                         key=lambda x: int(x.split('_')[1].split('.')[0]))

    # Tạo một canvas PDF
    c = canvas.Canvas(output_pdf)

    # Lặp qua từng file ảnh
    for image_file in tqdm(image_files, desc="Merging images"):
        img_path = os.path.join(input_dir, image_file)
        img = Image.open(img_path)
        # Lấy kích thước ảnh
        width, height = img.size
        # Chuyển đổi pixel sang inch (72 pixels = 1 inch trong PDF)
        width_inch = width / 72
        height_inch = height / 72
        # Đặt kích thước trang PDF bằng với kích thước ảnh
        c.setPageSize((width_inch * inch, height_inch * inch))
        # Vẽ ảnh lên trang
        c.drawImage(ImageReader(img), 0, 0, width=width_inch * inch, height=height_inch * inch)
        # Chuyển sang trang mới
        c.showPage()
    # Lưu PDF
    c.save()

# Thư mục chứa ảnh đã tải về
input_dir = "downloaded_images"

# Tên file PDF đầu ra
output_pdf = "merged_document.pdf"

print(f"Bắt đầu ghép {len(os.listdir(input_dir))} ảnh thành PDF...")
merge_images_to_pdf(input_dir, output_pdf)
print(f"Đã hoàn thành! File PDF được lưu tại: {output_pdf}")

Bắt đầu ghép 114 ảnh thành PDF...


Merging images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [01:28<00:00,  1.29it/s]


Đã hoàn thành! File PDF được lưu tại: merged_document.pdf
